In [ ]:
import numpy as np
import pandas as pd
import ast

In [ ]:
movies= pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
credits.head(1)

In [ ]:
# <<<<<<<<<<if you want specifically see the data of any one columns in array form >>>>>>>
credits.head(1)['crew'].values

In [ ]:
## now merging both the data frame because working with diff diff data is so relibable

In [ ]:
# now assigning new merge data to movies variable that is our main df
movies= movies.merge(credits, on='title')   

In [ ]:
# movies.shape  ---> this give the error becuase we reassign new movie pls run merge only one times next time your data that you 
# assign the movies have only those attributes that you wanted to merge them

In [ ]:
credits.shape

In [ ]:
movies.head(1)

In [ ]:
movies.shape
# columns is 23 because we merge on title so we do not add one more title on merge data

In [ ]:
# # important thing from df that we should create tag so we always focus on string attributes and avoid numeric
#  1. genres
#  2. id
#  3. keywords
#  4. title
#  5. overview
# 'releasing date' is important factor but again its numerical so it's not compatiable with our approcach
# 6. cast
# 7. crew
movies[['movie_id', 'title', 'overview', 'genres','keywords','cast', 'crew']]
# movies.head()

In [ ]:
# movies['original_language'].value_counts()    ##orginal_languange does not matter because highly imbalalced. eng is most used
movies.info()

In [ ]:
# now we start preprocessing of massy last columns 
movies.isnull().sum()

In [ ]:
# now we have only 3 missing data of "overview" row so we drop off
movies.dropna(inplace=True)

In [ ]:
# now check for duplicate data
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres
# so it is dictionary in list format but what I want to like list only

In [ ]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# ['Action', 'Adventure','Fantasy', 'Science Fiction']   isme lekar aana hai

In [ ]:
def convert(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        L.append(i['name'])
        
    return L
        

In [ ]:
movies['genres']= movies['genres'].apply(convert)

In [ ]:
movies.head()
# now the genres problem is solved now apply this same method for keywords also

In [ ]:
movies['keywords']= movies['keywords'].apply(convert)

In [ ]:
movies['cast'][0]
# now we take only top 3 cast in our data frame so

In [ ]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L
        

In [ ]:
movies['cast']= movies['cast'].apply(convert3)

In [ ]:
movies.head()

In [ ]:
# now we extract job dic in crew column cos we need only job section name of director.
import ast
def fetch_director(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
        break
    return L
        

In [ ]:
movies['crew'] =movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
# now we convert "overview " in series so that we easly concatenate with others.
movies['overview'][0]

In [ ]:
movies['overview']= movies['overview'].apply(lambda x :x.split())

In [ ]:
movies.head()

# now Remove the space " " between words in all columns 

In [ ]:
movies['genres'] =movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies.head()

In [ ]:
# now repeat above method for all columns
movies['keywords'] =movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] =movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] =movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


In [ ]:
movies.head()

In [ ]:
# now Concatenate the below columns
movies['tags']=movies['overview']+ movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
# now we don't need all columns so we make new data frame where we take only tags column instead of last 4 ecxet tags
new_df =movies[['movie_id','title', 'tags']]

In [ ]:
# now we convert the series of tags into string
new_df['tags']= new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'][0]

In [ ]:
# now we convert its lowercase letter cos its recommneded for tags
new_df['tags']= new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
# so now here we use steming from nltk library is toolkit for appliying static in natural pricessing language (NPL)
# why--> {love, loving, loved } that is one word love but diff form is problematic so we want to reduce the retundency to impove tags efficiency so
# here we use stemping to convert all word in one {love}

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
new_df['tags']= new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'][1]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv =CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector= cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vector.shape

In [ ]:
vector

In [ ]:
vector[0]

In [ ]:
# cv have lots of function where cv.get_feature_names show us all 5000 words that come out from tags corps(group of lots of string ).
cv.get_feature_names()

In [ ]:
ps.stem('love')

In [ ]:
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver')

In [ ]:
# 1) now next step you have to find the cosine distance between vector of tags rather than uklerian distance btw vector cos cosine dista
# is more efficient 
# 2) distance inversly propostional to similarities of movie

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# here we use similar_movie varible instead of similarity varible acc to campusx video tut youtbe
similar_movie =cosine_similarity(vector)

In [ ]:
#  This enumerated object can then be used directly for loops or converted into a list of tuples using the list() method.
sorted(list(enumerate(similar_movie[0])),reverse=True, key=lambda x:x[1])[1:6]

In [ ]:
# now we make actual recommended function which give you to five top similar movie 

In [ ]:
def recommend(movie):
    movie_index =new_df[new_df['title']==movie].index[0]
    distances =similar_movie[movie_index]
    movie_list =sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movie_list:
            print(new_df.iloc[i[0]].title)
          

In [ ]:
recommend('Batman Begins')

In [ ]:
# now we dump in pickle file
import pickle

In [ ]:
new_df['title'].values[1]

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
new_df.to_dict()

In [ ]:
# now we need similarity_movie in my mian app so we dump it in pickle file
pickle.dump(similar_movie, open('similar_movie.pkl', 'wb'))